# Perceptron

In [12]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

### 1) Подготовка датасета

In [24]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1,), (0.1,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

### 2) Нейронная сеть

In [25]:
class Perceptron(nn.Module):
  def __init__(self):
    super(Perceptron, self).__init__()
    self.fc1 = nn.Linear(28 * 28, 256)
    self.bn1 = nn.BatchNorm1d(256)
    self.fc2 = nn.Linear(256, 128)
    self.bn2 = nn.BatchNorm1d(128)
    self.fc3 = nn.Linear(128, 10)
    self.dropout = nn.Dropout(0.5)

  def forward(self, x):
    x = x.view(-1, 28 * 28)
    x = self.bn1(torch.relu(self.fc1(x)))
    x = self.dropout(x)
    x = self.bn2(torch.relu(self.fc2(x)))
    x = self.dropout(x)
    x = self.fc3(x)
    return x

model = Perceptron().to('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)

### 3) Тестирование модели

In [26]:
def evaluate(model, loader, device):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  return correct / total

### 4) Ранняя остановка

In [27]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0

    def should_stop(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

### 5) ModelCheckpoint

In [28]:
class ModelCheckpoint:
    def __init__(self, filepath, monitor='val_loss', mode='min', verbose=True):
      self.filepath = filepath
      self.monitor = monitor
      self.mode = mode
      self.best_score = None
      self.verbose = verbose

    def save_checkpoint(self, model, epoch, val_score):
        if self.best_score is None or (
            self.mode == 'min' and val_score < self.best_score or
            self.mode == 'max' and val_score > self.best_score
        ):
            self.best_score = val_score
            torch.save(model.state_dict(), self.filepath)
            if self.verbose:
                print(f"✅ Модель сохранена на эпохе {epoch+1} с {self.monitor}: {val_score:.4f}")


### 6) Тренировка и тестирование модели

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
early_stopping = EarlyStopping(patience=5)
checkpoint = ModelCheckpoint(filepath='best_model.pth', monitor='val_loss', mode='min', verbose=True)


for epoch in range(50):
  model.train()
  running_loss = 0.0
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  avg_train_loss = running_loss / len(train_loader)
  val_accuracy = evaluate(model, test_loader, device)

  print(f"Эпоха [{epoch+1}/{50}], Потери: {avg_train_loss:.4f}, Точность на валидации: {val_accuracy:.4f}")

  checkpoint.save_checkpoint(model, epoch, avg_train_loss)

  if early_stopping.should_stop(avg_train_loss):
    print("Ранняя остановка сработала.")
    break

model.load_state_dict(torch.load('best_model.pth'))

test_accuracy = evaluate(model, test_loader, device)
print(f"Точность на тесте: {test_accuracy:.4f}")

Эпоха [1/50], Потери: 0.7503, Точность на валидации: 0.7365
✅ Модель сохранена на эпохе 1 с val_loss: 0.7503
Эпоха [2/50], Потери: 0.7465, Точность на валидации: 0.7535
✅ Модель сохранена на эпохе 2 с val_loss: 0.7465
Эпоха [3/50], Потери: 0.7461, Точность на валидации: 0.6984
✅ Модель сохранена на эпохе 3 с val_loss: 0.7461
Эпоха [4/50], Потери: 0.7480, Точность на валидации: 0.7690
Эпоха [5/50], Потери: 0.7420, Точность на валидации: 0.7297
✅ Модель сохранена на эпохе 5 с val_loss: 0.7420
Эпоха [6/50], Потери: 0.7490, Точность на валидации: 0.7538
Эпоха [7/50], Потери: 0.7464, Точность на валидации: 0.7039
Эпоха [8/50], Потери: 0.7440, Точность на валидации: 0.7355
Эпоха [9/50], Потери: 0.7441, Точность на валидации: 0.6839
Эпоха [10/50], Потери: 0.7480, Точность на валидации: 0.7752
Ранняя остановка сработала.
Точность на тесте: 0.7297


### 7) Tensorflow Keras

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model

In [23]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

def create_model():
    inputs = Input(shape=(28 * 28,))

    x = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    outputs = Dense(10, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

model = create_model()
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

history = model.fit(
    x_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Точность на тесте: {test_accuracy:.4f}")


Epoch 1/50
1494/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7105 - loss: 2.8838
Epoch 1: val_loss improved from inf to 1.62541, saving model to best_model.h5


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.7106 - loss: 2.8809 - val_accuracy: 0.7599 - val_loss: 1.6254
Epoch 2/50
1494/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7198 - loss: 1.8699
Epoch 2: val_loss improved from 1.62541 to 1.60891, saving model to best_model.h5


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.7198 - loss: 1.8697 - val_accuracy: 0.7346 - val_loss: 1.6089
Epoch 3/50
1496/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7167 - loss: 1.7701
Epoch 3: val_loss improved from 1.60891 to 1.47928, saving model to best_model.h5


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.7167 - loss: 1.7701 - val_accuracy: 0.8018 - val_loss: 1.4793
Epoch 4/50
1497/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7161 - loss: 1.7397
Epoch 4: val_loss improved from 1.47928 to 1.43482, saving model to best_model.h5


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.7161 - loss: 1.7397 - val_accuracy: 0.8174 - val_loss: 1.4348
Epoch 5/50
1498/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7207 - loss: 1.7100
Epoch 5: val_loss did not improve from 1.43482
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.7207 - loss: 1.7100 - val_accuracy: 0.7898 - val_loss: 1.5105
Epoch 6/50
1496/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7247 - loss: 1.6694
Epoch 6: val_loss improved from 1.43482 to 1.38958, saving model to best_model.h5


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.7247 - loss: 1.6694 - val_accuracy: 0.8109 - val_loss: 1.3896
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7215 - loss: 1.6568
Epoch 7: val_loss did not improve from 1.38958
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.7215 - loss: 1.6568 - val_accuracy: 0.7671 - val_loss: 1.5284
Epoch 8/50
1494/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7169 - loss: 1.6619
Epoch 8: val_loss did not improve from 1.38958
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.7169 - loss: 1.6620 - val_accuracy: 0.8163 - val_loss: 1.4026
Epoch 9/50
1499/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7166 - loss: 1.6800
Epoch 9: val_loss did not improve from 1.38958
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.7166 - loss: 1.6799 - val_accuracy: 0.8005 - val_loss: 1.4670
Epoch 10/50
1498/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7179 - loss: 1.6350
Epoch 10: val_loss did not